In [1]:
'''
06/06/25
for now this is a notebook, but if the plan is to automate,
it will become a .py file that can be executed by terminal
'''

# NB - HAS TROUBLE WRITING TO ONEDRIVE
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display, clear_output


def show_frame(frame):
    '''sort of obsolete. function to show video inline. but prefered 
    method is just writing a new video mp4.'''
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame_rgb)
    plt.axis('off')
    plt.show()

In [2]:
def midpoint(p1, p2):
    '''gets midpoint from two pairs of coords'''
    midp = np.array([(p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2])
    return midp


def get_COR(p1a, p2a, p1b, p2b, frame):
    '''1 and 2 refer to endpoints, a and b refer to times. '''
    # v1 and v2 are vectors between the two timelapses.
    v1 = p1b - p1a
    v2 = p2b - p2a

    # calculate the vector midpoint 
    m1 = midpoint(p1a, p1b)
    m2 = midpoint(p2a, p2b)
    
    # perpendicular bisector lines
    n1 = np.array([-v1[1], v1[0]])
    n2 = np.array([-v2[1], v2[0]])

    # builds matrix to solve linalg equation, form b = A @ x
    A = np.stack([n1, -n2], axis=1)
    b = m2 - m1
    
    # see lab book
    t = np.linalg.solve(A, b)
    centre = m1 + t[0] * n1

    # NEEDS TO BE INT FOR PLOTTING
    m1 = [int(m1[0]), int(m1[1])]
    m2 = [int(m2[0]), int(m2[1])]

    # How far to extend the perpendicular line
    scale_factor = 150 
    
    # For the first perpendicular bisector
    pt1pos = (
        m1[0] + int(n1[0] * scale_factor / np.linalg.norm(n1)),
        m1[1] + int(n1[1] * scale_factor / np.linalg.norm(n1))
    )
    pt1neg = (
        m1[0] - int(n1[0] * scale_factor / np.linalg.norm(n1)),
        m1[1] - int(n1[1] * scale_factor / np.linalg.norm(n1))
    )
    
    # For the second perpendicular bisector
    pt2pos = (
        m2[0] + int(n2[0] * scale_factor / np.linalg.norm(n2)),
        m2[1] + int(n2[1] * scale_factor / np.linalg.norm(n2))
    )
    pt2neg = (
        m2[0] - int(n2[0] * scale_factor / np.linalg.norm(n2)),
        m2[1] - int(n2[1] * scale_factor / np.linalg.norm(n2))
    )
    
    # draw line 1
    cv2.line(frame, pt1pos, pt1neg, (255, 0, 0), 1)
    cv2.line(frame, pt2pos, pt2neg, (255, 0, 0), 1)
    
    # Draw from midpoint to extended points
    cv2.line(frame, m1, pt1pos, (255, 0, 0), 1)
    cv2.line(frame, m1, pt1neg, (255, 0, 0), 1)
    
    return (int(centre[0]), int(centre[1]))


def main_COR(inpath, outpath, skip_frame=1, start_time=0, validate=False):
    '''input: file to analyse, output: path to analysed video. optional:
    number of frames to skip to get centre, seconds of video to skip.'''
    # loading input
    cap = cv2.VideoCapture(inpath)
    
    # matches the new video to the old video specifications
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # skips ahead ||| MAKE SURE PROBE IS VISIBLE FROM THE FILM START ||| big obstacle for automation
    start_frame = int(start_time * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # saving output to mp4 format
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_out = cv2.VideoWriter(outpath, fourcc, fps, (width, height))
    if not video_out.isOpened():
        print("main_COR: VideoWriter failed to open.")
        return  # exits function

    # needed for rotational speed calc
    frame_info = []
    rot_list = []
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        # end of video
        if not ret:
            break
        
        # filtering for red and then using 
        mask = colour_mask(frame, colour='red')
        p1, p2 = detect_line(mask)
        
        if p1 and p2 is not None:
            p1 = np.array(p1)
            p2 = np.array(p2)

            # just for tracking really not needed
            mid = midpoint(p1, p2)

            # storing this info for use in next part
            frame_info.append((frame_count, p1, p2))
            
# ============ COMPARING WITH PREVIOUS FRAMES TO FIND COR ==========
# A and B refer to time, 1 and 2 to endpoints again
            if len(frame_info) >= skip_frame + 1:
                # takes the info stored in the frame one skip ago
                frameA_count, frameA_1, frameA_2 = \
                    frame_info[-(skip_frame + 1)]
                
                # compares to last frame
                frameB_count, frameB_1, frameB_2 = \
                    frame_info[-1]

                # p1a, p2a, p1b, p2b
                centre = get_COR(frameA_1, frameA_2, frameB_1, frameB_2, frame)

                if centre is not None:
                    rot_list.append(centre)

# cv2.circle(img, centre, radius, colour, thickness=-1 fills it)
# cv2.line(frame, p1, p2, (RGB), thickness)
                    # drawing previous line (darker red)
                    cv2.line(frame, frameA_1, frameA_2, (0, 0, 165), 2)
                    # midpoint
                    cv2.circle(frame, tuple(mid.astype(int)), 4, 
                                (0, 225, 225), -1)
                    # COR
                    cv2.circle(frame, centre, 6,
                                (0, 255, 0), 4)
                    # mapping lines
                    cv2.line(frame, frameA_1, frameB_1, (0, 255, 0), 1)
                    cv2.line(frame, frameA_2, frameB_2, (0, 255, 0), 1)
                    # perpendicular bisectors

                else:
                    print("get_COR returned None")
                    exit()
                    
            video_out.write(frame)

            frame_count = frame_count + 1
    
    video_out.release()
    print(f"Video saved as {outpath}")
    
    centres = np.array(rot_list)
    try:
        mean_COR = centres.mean(axis=0).astype(int)
        print("Mean centre vs true centre = ",
              (mean_COR[0] - centre[0]), (mean_COR[1] - centre[1]) )

    except RuntimeWarning:
        pass

    if validate is True:
        return centres
    

def colour_mask(frame, colour="red"):
    '''input: frame. returns: mask. Potential to have HSV lookup table
    for different colours, but we will probably use red.'''
    # threshold for red in HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    if colour == "red":
        # red wraps round two bands of the HSV spectrum
        lower_red1 = np.array([0, 100, 100])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160, 100, 100])
        upper_red2 = np.array([180, 255, 255])
        
        # turns all pixels to black if not in the HSV red ranges
        # and combines the two masks
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask = cv2.bitwise_or(mask1, mask2)
        
        return mask
    
    elif colour == "blue":
        # these HSV values work really well for the probe colour
        lower_blue = np.array([100, 150, 100])  # H, S, V
        upper_blue = np.array([130, 255, 255])
        mask = cv2.inRange(hsv, lower_blue, upper_blue)
        
        return mask


def detect_line(mask):
    '''input: mask. returns: tuple, tuple. If lines not none,
    returns point 1 and 2 coords.'''
    # canny removes noise with a 5x5 gaussian filter,
    # converts a normal image into a sketch made up of lines only.
    edges = cv2.Canny(mask, 50, 150)  # args min and max val - threshold for edge
    
    # line detection - Hough algorithm probabilistic
    # OR with high contrast - use cv2.findNonZero() to directly fit a line
    # w/ cv2.fitLine
    # =======================================================
    # HoughLines expresses line in polar coords. Intersections define points
    # belonging to the same line. Threshold is min intersections to define line
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50,
                            minLineLength=50, maxLineGap=10)

# lines is an array([[[x1, y1, x2, y2]],
#                    [[x1, y1, x2, y2]])
# lines[0][0] is = line[0]

    if lines is not None:
        # finding the longest line and returning the end points
        # again probably overkill for one line as long as the mask works
        longest = None        
        max_len = 0
        for line in lines:
        # line is lines[i] and line[0] just extracts [x1, y1, x2, y2]
            x1, y1, x2, y2 = line[0]
            # length = np.hypot(x2 - x1, y2 - y1)
            length = np.sqrt( (x2 - x1)**2 + (y2 - y1)**2)
            if length > max_len:
                max_len = length
                longest = line[0]

        if longest is not None:
            x1, y1, x2, y2 = longest
            return [x1, y1], [x2, y2]
        
    return None, None  # no lines found




In [3]:
def mask_check(inpath, outpath, start_time=0, colour='red'):
    '''input: file to analyse, output: path to analysed video. video of mask.'''
    # loading input
    cap = cv2.VideoCapture(inpath)
    
    # matches the new video to the old video specifications
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # skips ahead ||| MAKE SURE PROBE IS VISIBLE FROM THE FILM START |||
    start_frame = int(start_time * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # saving output to mp4 format
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_out = cv2.VideoWriter(outpath, fourcc, fps, (width, height))
    if not video_out.isOpened():
        print("mask_check: VideoWriter failed to open.")
        return
    
    while cap.isOpened():
        ret, frame = cap.read()
        # end of video
        if not ret:
            break
        
        # filtering for red and then using 
        mask = colour_mask(frame, colour=colour)

        video_out.write(mask)

    cap.release()
    video_out.release()
    print(f"Video saved as {outpath}")
    
    



In [ ]:
# input = "M3B_EXTENDED_UR1_154519.mp4"
# # input = r"C:\Users\rj429\test_line_video.mp4"
# output = r"C:\Users\rj429\analysed_COR_video_blue.mp4"

input = r"C:\Users\rj429\test_line_video.mp4"
ifile = r"C:\Users\rj429\M1B_R1S_145621480.mp4"
# i = np.random.randint(1,100)
# print(i)
output = f"C:\\Users\\rj429\\result_1.mp4"
# bool = test_data(input, amplitude=0)
# mask_check(ifile, output, start_time=11)
# test_data(input, centre=[205, 235], thickness=2, amplitude=0)
main_COR(input, output, start_time=0)
# validate(input, output, amplitude=0)




Video saved as C:\Users\rj429\test_line_video.mp4
Video saved as C:\Users\rj429\result_1.mp4
Mean centre vs true centre =  8 17


In [ ]:
import cv2
import numpy as np

# Load image and convert to grayscale
input = "M3R2S_155427.mp4"

output = "rectangle_test_vid.mp4"

cap = cv2.VideoCapture(input)
fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frames_to_show = []

# saving output to mp4 format
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_out = cv2.VideoWriter(output, fourcc, fps, (width, height))
if not video_out.isOpened():
    print("mask_check: VideoWriter failed to open.")
    exit()

# skips some time ahead to stable rotation
start_time = 10
end_time = 15
start_frame = int(start_time * fps)
end_frame = int(end_time * fps)
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

frame_count = 0

while cap.isOpened() and frame_count <= end_frame:
    ret, frame = cap.read()
    if not ret:
        break

    mask = colour_mask(frame)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Approximate contour to polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Filter polygons with 4 vertices (possible rectangles)
        if len(approx) == 4:
            # Check if polygon is convex
            if cv2.isContourConvex(approx):
                # Optionally: check aspect ratio and angles
                # Extract points
                pts = approx.reshape(4, 2)
                cv2.drawContours(frame, [approx], -1, (0, 255, 0), 3)
    

                
                
                # # Compute aspect ratio
                # (x, y, w, h) = cv2.boundingRect(approx)
                # aspect_ratio = float(w) / h
                
                # # Check aspect ratio if you want rectangles not squares, e.g.:
                # if 0.8 < aspect_ratio < 6:  # roughly square, adjust as needed
                #     # Optional: check angles ~90 degrees
                #     def angle(pt1, pt2, pt0):
                #         dx1 = pt1[0] - pt0[0]
                #         dy1 = pt1[1] - pt0[1]
                #         dx2 = pt2[0] - pt0[0]
                #         dy2 = pt2[1] - pt0[1]
                #         return abs(np.arccos((dx1*dx2 + dy1*dy2) / 
                #                     (np.sqrt(dx1**2 + dy1**2) * np.sqrt(dx2**2 + dy2**2))) * 180 / np.pi)
                    
                #     angles = []
                #     for i in range(4):
                #         angles.append(angle(pts[i], pts[(i-2)%4], pts[(i-1)%4]))
                #     if all(80 < a < 100 for a in angles):
                #         cv2.drawContours(frame, [approx], -1, (0, 255, 0), 3)
    
    video_out.write(frame)
    frame_count = frame_count + 1


cap.release()
video_out.release()



In [26]:
mask_check("M1B_UR1S_145845345.mp4", "rectangle_mask.mp4", start_time=5)

Video saved as rectangle_mask.mp4


In [3]:
video_in = "M3R2S_155427.mp4"


def line_orientation(contour):
    ''''''
    data_pts = np.array(contour, dtype=np.float32).reshape(-1, 2)  #

    # 
    mean, eigenvectors = cv2.PCACompute(data_pts, mean=None)

    # eigenvectors doesn't always have the right shape. Possibly due to no contours found.
    # but throws errors and stops code for any failure otherwise
    if eigenvectors is not None and eigenvectors.shape[0] >= 1 and eigenvectors.shape[1] >= 2:
        #
        center = tuple(mean[0])
        
        # angle = arctan(y/x)
        angle = np.arctan2(eigenvectors[0, 1], eigenvectors[0, 0])  # radians

        return center, angle
        
    else:
        return None, None


def main()
cap = cv2.VideoCapture(video_in)
fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second

# skips some time ahead to stable rotation
start_time = 10
start_frame = int(start_time * fps)
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

frames_to_show = []

# initialised for appending values for each frame and counting frames
angles = []
timestamps = []
frame_number = 0



while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    LOWER_BLUE = np.array([100, 150, 100])  # H, S, V
    UPPER_BLUE = np.array([130, 255, 255])
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # blue_mask = cv2.inRange(hsv, LOWER_BLUE, UPPER_BLUE)
    mask = colour_mask(hsv)
    
#     plt.imshow(blue_mask, cmap='gray')  # Use gray colormap for binary masks
#     plt.title("Blue Mask")
#     plt.axis('off')
#     plt.show()
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        largest = max(contours, key=cv2.contourArea)
        center, angle_rad = get_orientation(largest)
        if center is None or angle_rad is None:
            continue  # Skip this frame
        else:
            angles.append(angle_rad)
            timestamps.append((frame_number - start_frame) / fps)

        # Convert center to int for drawing
        center_int = tuple(map(int, center))

        # defines endpoint of line that runs through the geometric centre
        # found by contours
        length = 150
        x1 = int(center[0] - length/2 * np.cos(angle_rad))
        y1 = int(center[1] - length/2 * np.sin(angle_rad))
        x2 = int(center[0] + length/2 * np.cos(angle_rad))
        y2 = int(center[1] + length/2 * np.sin(angle_rad))


        # Draw the major axis line
        cv2.line(frame, center_int, (x2, y2), (0, 255, 0), 2)  # green line

        # Optionally draw the center point
        cv2.circle(frame, center_int, 5, (0, 0, 255), -1)  # red dot
        
    # Convert BGR to RGB for matplotlib
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames_to_show.append(frame_rgb)

    frame_number = frame_number + 1
    

angles = np.unwrap(angles)  # unwrap to avoid 360° jump
angular_velocity = np.gradient(angles, timestamps)  # radians per second

# Plot
plt.plot(timestamps, angular_velocity)
plt.xlabel('Time (s)')
plt.ylabel('Angular velocity (rad/s)')
plt.title('Rotational Velocity Over Time')
plt.show()

cap.release()

# matplotlib to create HTML to watch video inline
fig, ax = plt.subplots()
im = ax.imshow(frames_to_show[0])
plt.axis('off')


def update(i):
    im.set_array(frames_to_show[i])
    return [im]

ani = FuncAnimation(fig, update, frames=len(frames_to_show), interval=800 / fps, blit=True)

HTML(ani.to_html5_video())


SyntaxError: expected ':' (3754308949.py, line 26)